# 20.0. Data Enrichment - Integrated Dataset - OTX Pulses

In [1]:
pip install OTXv2

Note: you may need to restart the kernel to use updated packages.


In [2]:
entries_dir      = "C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\Integrated_Dataset\\Entries.csv"
attacks_dir      = "C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\Integrated_Dataset\\Attacks.csv"
anomalies_dir    = "C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\Integrated_Dataset\\Anomalies.csv"
tv_analysis_dir  = "C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\Integrated_Dataset\\TV_Analysis.csv"
behaviours_dir   = "C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\Integrated_Dataset\\Behaviours.csv"
yaraRules_dir    = "C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\Integrated_Dataset\\Yara_Rules.csv"
tags_dir         = "C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataCleaning\\Integrated_Dataset\\Tags.csv"

In [3]:
from datetime import datetime
import os
import time
import pandas as pd 
import requests
import urllib3
import json
import sys
import numpy as np
import pandas as pd
from OTXv2 import OTXv2

pd.options.display.max_colwidth = 1000
pd.set_option('display.max_columns', None)

In [4]:
api_key = '006ee711b0794e449a6d074aacdda04a285032c7e787893b095cc8a2308f99ee'
otx = OTXv2('006ee711b0794e449a6d074aacdda04a285032c7e787893b095cc8a2308f99ee')

## Entries Enrichment 
Add the # pulses in which this file is reported

In [5]:
entries_df = pd.read_csv(entries_dir, low_memory = False)
entries_df

,sha1,sha256,md5,first_seen,last_seen,analysis_date,file_type,delivery_method
0,411535c9548f86034a6a9a603de521a86a3b5466,04c53261b1220a894a02f5ffb39cdfd73f93481c0b5c8106d21c91b20205c62d,5b5a0225a2b7b9a054417662c733168a,2021-11-09,2021-11-09,2021-11-09,exe,email attachment
1,1ad9e9761fd6935c0cf5048c9615d0383baac48e,ac901bf5882f14e9e07235b8488b6479b4519addda6dbfb89147401c1e9e6e4f,da9534900ee0d11c9b30cf33152ea03c,2023-07-19,2023-08-25,2023-07-19,exe,web download
2,8ab63abfe58fb218cc001581ac0d9fe38c784157,7f514ed5e1ec262953e6252a4089531c519e95d700c5808415b0f049fc59a5f0,855232a26e8afcd0cca5f816a1919dee,2022-01-04,2022-02-04,2022-01-04,exe,unknown
3,55cbf8dae95d350ad7e5a63b418f6cd203488f51,8b7ae9f195b075a789d6d8277d500d27754bfa3c53ecca8db7beac8ccd07884f,97d9d216f2627cb238ca6637580b1739,2023-06-13,2023-06-13,2022-02-09,exe,unknown
4,a2915c1be9e6134b7bf3ca5ca00eeb0c969bedab,6c2a2251861a6d2701814843fadac940cf4d34db9f446f0698352fd866b31739,1b09de36dfe5850d7e3fbd6b39c89a43,2022-03-27,2022-04-06,2021-02-11,exe,web download
...,...,...,...,...,...,...,...,...
2657,6b6955af591154d3f6e2db35ca9904cce7e264b1,3a9d8c693b7dc6a9c2bf876c5626ebd0923e35bd5f698baed248df7f51f67275,03872a722df0b073a84fd1aa45eb9376,NaN,NaN,2020-02-20,elf,unknown
2658,3ab53037e2360c42b57d213234ced58717844a71,5de4932cb0ccab3fec711135cdd1af5a09c26a870bfb2d2adc650d0fdbf16488,e0fea510c784eb0e440e99e53d83d414,NaN,NaN,2021-07-14,exe,unknown
2659,4f9b9859a596a8bfdfee516788587ca0c0fb6ae8,f10dd5ce32d72489dd2696e5b58ee115f3cd065969a61a6613cab27bcd57de70,c9f771be253b6ca1d7a808d7d65c5e29,NaN,NaN,2020-09-08,exe,unknown
2660,d14540087dd6d350fb49e644e53ccefc0e4c1582,dba8a4717d516772b8cc06bd258265f21f7db6ddab62328917524c5a243c67a1,fe783ff4bccd76cb215ee1f7738d9ed5,NaN,NaN,2021-12-20,exe,unknown


#### Example

In [6]:
i = 9
h = '04c53261b1220a894a02f5ffb39cdfd73f93481c0b5c8106d21c91b20205c62d'
sha1 = '411535c9548f86034a6a9a603de521a86a3b5466'

tmp1 = pd.DataFrame()

otx_url = 'https://otx.alienvault.com/api/v1/indicators/file/'+ sha1

current_pulse = requests.get(otx_url,headers={"X-OTX-API-KEY":api_key})

current_pulse = json.loads(current_pulse.content.decode("utf-8", "ignore"))

tmp = pd.DataFrame.from_dict(pd.json_normalize(current_pulse)).reset_index(drop = True)

tmp

,sections,type,type_title,indicator,validation,false_positive,base_indicator.id,base_indicator.indicator,base_indicator.type,base_indicator.title,base_indicator.description,base_indicator.content,base_indicator.access_type,base_indicator.access_reason,pulse_info.count,pulse_info.pulses,pulse_info.references,pulse_info.related.alienvault.adversary,pulse_info.related.alienvault.malware_families,pulse_info.related.alienvault.industries,pulse_info.related.other.adversary,pulse_info.related.other.malware_families,pulse_info.related.other.industries
0,"[general, analysis]",sha1,FileHash-SHA1,411535c9548f86034a6a9a603de521a86a3b5466,[],[],3291621002,5b5a0225a2b7b9a054417662c733168a,FileHash-MD5,,,,public,,4,"[{'id': '65709aa7e685939cb8ce7484', 'name': 'Malware Bazaar 4', 'description': '', 'modified': '2023-12-06T16:00:39.550000', 'created': '2023-12-06T16:00:39.550000', 'tags': [], 'references': [], 'public': 1, 'adversary': '', 'targeted_countries': [], 'malware_families': [], 'attack_ids': [], 'industries': [], 'TLP': 'green', 'cloned_from': None, 'export_count': 1, 'upvotes_count': 0, 'downvotes_count': 0, 'votes_count': 0, 'locked': False, 'pulse_source': 'api', 'validator_count': 0, 'comment_count': 0, 'follower_count': 0, 'vote': 0, 'author': {'username': 'StreamMiningEx', 'id': '262917', 'avatar_url': 'https://otx.alienvault.com/assets/images/default-avatar.png', 'is_subscribed': False, 'is_following': False}, 'indicator_type_counts': {'FileHash-SHA256': 52080, 'FileHash-MD5': 10295, 'FileHash-SHA1': 10295}, 'indicator_count': 72670, 'is_author': False, 'is_subscribing': None, 'subscriber_count': 18, 'modified_text': '38 days ago ', 'is_modified': False, 'groups': [], 'in_group...",[https://threatfox.abuse.ch/],[],[],[],[],[],[]


In [7]:
i = 9
h = '04c53261b1220a894a02f5ffb39cdfd73f93481c0b5c8106d21c91b20205c62d'
sha1 = '411535c9548f86034a6a9a603de521a86a3b5466'
md5 = '5b5a0225a2b7b9a054417662c733168a'

tmp1 = pd.DataFrame()

otx_url = 'https://otx.alienvault.com/api/v1/indicators/file/'+ sha1

current_pulse = requests.get(otx_url,headers={"X-OTX-API-KEY":api_key})

current_pulse = json.loads(current_pulse.content.decode("utf-8", "ignore"))

tmp = pd.DataFrame.from_dict(pd.json_normalize(current_pulse)).reset_index(drop = True)

if ('pulse_info.count' in tmp.columns) & ('pulse_info.pulses' in tmp.columns):

    if tmp.at[0,'pulse_info.count'] > 0:
        
        tmp1 = pd.json_normalize(pd.DataFrame.from_dict(pd.json_normalize(tmp['pulse_info.pulses']).T).reset_index(drop = True)[0])
        
tmp1     


 

,id,name,description,modified,created,tags,references,public,adversary,targeted_countries,malware_families,attack_ids,industries,TLP,cloned_from,export_count,upvotes_count,downvotes_count,votes_count,locked,pulse_source,validator_count,comment_count,follower_count,vote,indicator_count,is_author,is_subscribing,subscriber_count,modified_text,is_modified,groups,in_group,threat_hunter_scannable,threat_hunter_has_agents,related_indicator_type,related_indicator_is_active,author.username,author.id,author.avatar_url,author.is_subscribed,author.is_following,indicator_type_counts.FileHash-SHA256,indicator_type_counts.FileHash-MD5,indicator_type_counts.FileHash-SHA1,indicator_type_counts.hostname,indicator_type_counts.domain,indicator_type_counts.URL
0,65709aa7e685939cb8ce7484,Malware Bazaar 4,,2023-12-06T16:00:39.550000,2023-12-06T16:00:39.550000,[],[],1,,[],[],[],[],green,None,1,0,0,0,False,api,0,0,0,0,72670,False,None,18,38 days ago,False,[],False,True,1,FileHash-SHA256,1,StreamMiningEx,262917,https://otx.alienvault.com/assets/images/default-avatar.png,False,False,52080,10295,10295,NaN,NaN,NaN
1,65707ef8b5b5810d62645bfd,C2 Servers & Virus Providers & Malware Hashes,,2023-12-06T14:02:32.818000,2023-12-06T14:02:32.818000,[],[],1,,[],[],[],[],green,None,0,0,0,0,False,api,0,0,0,0,8410,False,None,18,38 days ago,False,[],False,True,1,FileHash-SHA1,1,StreamMiningEx,262917,https://otx.alienvault.com/assets/images/default-avatar.png,False,False,2950,591,573,183.0,1198.0,2915.0
2,645b97eb9e10e5961a10455e,Malware Bazaar 4,,2023-05-10T13:11:07.350000,2023-05-10T13:11:07.350000,[],[],1,,[],[],[],[],white,None,1,0,0,0,False,web,0,0,0,0,72670,False,None,16,248 days ago,False,[],False,True,1,FileHash-SHA256,1,LoveAndren,222280,https://otx.alienvault.com/assets/images/default-avatar.png,False,False,52080,10295,10295,NaN,NaN,NaN
3,6215ef048fbebbb7f207044d,C2 Servers & Virus Providers & Malware Hashes,,2022-03-25T00:03:52.440000,2022-02-23T08:23:32.389000,[],[https://threatfox.abuse.ch/],1,,[],[],[],[],white,None,7,0,0,0,False,web,0,0,0,0,8410,False,None,85,659 days ago,True,[],False,True,1,FileHash-SHA1,1,bluewatcher,174522,https://otx.alienvault.com/assets/images/default-avatar.png,False,False,2950,591,573,183.0,1198.0,2915.0


In [8]:
usefull_columns = ['id', 'name', 'description', 'modified', 'TLP', 'upvotes_count', 'downvotes_count', 'author.username']

#### Data collection PULSES

In [9]:
def create_otx_pulses_tdf(df):
    
    max_retries = 5
    retry_delay= 10
    dataframe = pd.DataFrame()
    
    for i in range(len(df)):
        
        print(i)
        
        sha256 = df.at[i, 'sha256']
        sha1 = df.at[i, 'sha1']
        md5 = df.at[i, 'md5']
        
        first_seen = df.at[i, 'first_seen']
        last_seen = df.at[i, 'last_seen']
        
        otx_url = 'https://otx.alienvault.com/api/v1/indicators/file/'+ str(sha256)
        
        try:
            for attempt in range(1, max_retries + 1):

                current_pulse = requests.get(otx_url,headers={"X-OTX-API-KEY":api_key})

                current_pulse = json.loads(current_pulse.content.decode("utf-8", "ignore"))

                tmp = pd.DataFrame.from_dict(pd.json_normalize(current_pulse)).reset_index(drop = True)
                
                if ('base_indicator.type' in tmp.columns) & ('base_indicator.indicator' in tmp.columns):
                    
                    found = False
                    
                    indicator_type = tmp.at[0, 'base_indicator.type']
                    indicator_base = tmp.at[0, 'base_indicator.indicator']
                    
                    if (indicator_type == 'FileHash-MD5') & ( indicator_base == md5 ):
                        found = True
                        
                    elif (indicator_type == 'FileHash-SHA1') & ( indicator_base == sha1 ): 
                        found = True
                        
                    if(found):
                        if ('pulse_info.count' in tmp.columns) & ('pulse_info.pulses' in tmp.columns):

                            if tmp.at[0,'pulse_info.count'] > 0:

                                tmp1 = pd.json_normalize(pd.DataFrame.from_dict(pd.json_normalize(tmp['pulse_info.pulses']).T).reset_index(drop = True)[0])

                                tmp1['sha256'] = sha256
                                tmp1['sha1'] = sha1
                                tmp1['md5'] = md5
                                
                                dataframe = pd.concat([dataframe, tmp1])
                                break
                                
                        else:
                            break
                                
                    else:
                        break
                        
                else:
                    break
                        
                
        except json.JSONDecodeError as e:
            print(f"JSON Decode Error on attempt {attempt}/{max_retries}: {e}")
            if attempt < max_retries:
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
                
            else:
                print("Max retries reached. Unable to handle JSON Decode Error.")
                
        except (requests.exceptions.ReadTimeout, requests.exceptions.RequestException) as e:
            print(f"Request attempt {attempt} timed out. Retrying...")
            if attempt < max_retries:
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)

            else:
                print("Max retries reached. Unable to handle JSON Decode Error.")
                
    dataframe['scanner_date'] = datetime.now()   
            
    return dataframe.reset_index(drop = True)         

In [10]:
otx_pulses_df = create_otx_pulses_tdf(entries_df)
otx_pulses_df

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,id,name,description,modified,created,tags,references,public,adversary,targeted_countries,malware_families,attack_ids,industries,TLP,cloned_from,export_count,upvotes_count,downvotes_count,votes_count,locked,pulse_source,validator_count,comment_count,follower_count,vote,indicator_count,is_author,is_subscribing,subscriber_count,modified_text,is_modified,groups,in_group,threat_hunter_scannable,threat_hunter_has_agents,related_indicator_type,related_indicator_is_active,author.username,author.id,author.avatar_url,author.is_subscribed,author.is_following,indicator_type_counts.FileHash-SHA256,indicator_type_counts.FileHash-MD5,indicator_type_counts.FileHash-SHA1,indicator_type_counts.hostname,indicator_type_counts.domain,indicator_type_counts.URL,sha256,sha1,md5,indicator_type_counts.CVE,indicator_type_counts.FileHash-IMPHASH,indicator_type_counts.URI,indicator_type_counts.FilePath,indicator_type_counts.Mutex,indicator_type_counts.YARA,indicator_type_counts.email,indicator_type_counts.IPv4,indicator_type_counts.CIDR,indicator_type_counts.IPv6,indicator_type_counts.SSLCertFingerprint,indicator_type_counts.JA3,scanner_date
0,65709aa7e685939cb8ce7484,Malware Bazaar 4,,2023-12-06T16:00:39.550000,2023-12-06T16:00:39.550000,[],[],1,,[],[],[],[],green,None,1,0,0,0,False,api,0,0,0,0,72670,False,None,18,38 days ago,False,[],False,True,1,FileHash-SHA256,1,StreamMiningEx,262917,https://otx.alienvault.com/assets/images/default-avatar.png,False,False,52080.0,10295.0,10295.0,NaN,NaN,NaN,04c53261b1220a894a02f5ffb39cdfd73f93481c0b5c8106d21c91b20205c62d,411535c9548f86034a6a9a603de521a86a3b5466,5b5a0225a2b7b9a054417662c733168a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-13 19:56:48.553500
1,65707ef8b5b5810d62645bfd,C2 Servers & Virus Providers & Malware Hashes,,2023-12-06T14:02:32.818000,2023-12-06T14:02:32.818000,[],[],1,,[],[],[],[],green,None,0,0,0,0,False,api,0,0,0,0,8410,False,None,18,38 days ago,False,[],False,True,1,FileHash-SHA1,1,StreamMiningEx,262917,https://otx.alienvault.com/assets/images/default-avatar.png,False,False,2950.0,591.0,573.0,183.0,1198.0,2915.0,04c53261b1220a894a02f5ffb39cdfd73f93481c0b5c8106d21c91b20205c62d,411535c9548f86034a6a9a603de521a86a3b5466,5b5a0225a2b7b9a054417662c733168a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-13 19:56:48.553500
2,645b97eb9e10e5961a10455e,Malware Bazaar 4,,2023-05-10T13:11:07.350000,2023-05-10T13:11:07.350000,[],[],1,,[],[],[],[],white,None,1,0,0,0,False,web,0,0,0,0,72670,False,None,16,248 days ago,False,[],False,True,1,FileHash-SHA256,1,LoveAndren,222280,https://otx.alienvault.com/assets/images/default-avatar.png,False,False,52080.0,10295.0,10295.0,NaN,NaN,NaN,04c53261b1220a894a02f5ffb39cdfd73f93481c0b5c8106d21c91b20205c62d,411535c9548f86034a6a9a603de521a86a3b5466,5b5a0225a2b7b9a054417662c733168a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-13 19:56:48.553500
3,6215ef048fbebbb7f207044d,C2 Servers & Virus Providers & Malware Hashes,,2022-03-25T00:03:52.440000,2022-02-23T08:23:32.389000,[],[https://threatfox.abuse.ch/],1,,[],[],[],[],white,None,7,0,0,0,False,web,0,0,0,0,8410,False,None,85,659 days ago,True,[],False,True,1,FileHash-SHA1,1,bluewatcher,174522,https://otx.alienvault.com/assets/images/default-avatar.png,False,False,2950.0,591.0,573.0,183.0,1198.0,2915.0,04c53261b1220a894a02f5ffb39cdfd73f93481c0b5c8106d21c91b20205c62d,411535c9548f86034a6a9a603de521a86a3b5466,5b5a0225a2b7b9a054417662c733168a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-13 19:56:48.553500
4,64e3a32396d5268bdc039d62,KataKobra Test Pulse,Test IOC,2023-08-21T17:47:15.188000,2023-08-21T17:47:15.188000,"[filehashsha256, filehashmd5, filehashsha1, sha1, indicator, cve20170199]",[64d6a03d4d231d4c32248dd4.csv],1,,[],[],[],[],white,None,4,0,0,0,False,web,0,0,0,0,379,False,None,1,145 days ago,False,[],False,True,1,FileHash-SHA1,1,katakobra,249457,https://otx.alienvault.com/assets/images/default-avatar.png,False,False,170.0,104.0,104.0,NaN,NaN,NaN,ac901bf5882f14e9e07235b8488b6479b4519add

In [11]:
otx_pulses_df.to_csv(r'C:\Users\leona\OneDrive\Desktop\Tesi\Pipeline\Datasets\Pipeline\FileAnalysis\PostDataEnrichment\Step2\OTXPulses.csv')